In [1]:
import sys, os
helper_py_location = os.path.abspath(os.path.join(os.path.join(os.getcwd(), "../..") ,"src","helper"))
print(helper_py_location)
sys.path.append(helper_py_location)
sys.path

e:\zhaoj\download\workSpace\Portfolio_Management\src\helper


['C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip',
 'C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312\\DLLs',
 'C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312\\Lib',
 'C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312',
 'e:\\zhaoj\\download\\workSpace\\.venv',
 '',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages\\win32',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages\\win32\\lib',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages\\Pythonwin',
 'e:\\zhaoj\\download\\workSpace\\Portfolio_Management\\src\\helper']

In [2]:
import helper
HF_returns_original = helper.load_data('HW_Hedge_Fund.xlsx',1)
M_ER_country_original = helper.load_data('HW_World.xlsx')
Emp_factors_original = helper.load_data('HW_Factors.xlsx')
HP_DJ_Sto_original = helper.load_data('HW_DJIA_Prices.xlsx',4)

#clean
HF_returns =HF_returns_original.copy()
Emp_factors = Emp_factors_original.copy()
M_ER_country = M_ER_country_original.copy()
HP_DJ_Sto = HP_DJ_Sto_original.copy()

Emp_factors.iloc[:, 1:] = Emp_factors.iloc[:, 1:].div(100)
M_ER_country.iloc[:,1:] = M_ER_country.iloc[:,1:].div(100)
Emp_factors.rename(columns={Emp_factors.keys()[0]: 'Date'}, inplace=True)

Part 3 Portfolio Construction
1. famlize the Dow Jones
2. famlize diff weighting methodologies

1. Different Portfolios - weights

In [5]:
print(type(HP_DJ_Sto))
HP_DJ_Sto.head()

<class 'pandas.core.frame.DataFrame'>


,Date,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,...,NKE,PG,CRM,BAC,UNH,VZ,HWP,PFE,WMT,DJIA
0,2004-06-30,90.01,51.38,54.57,49.0842,51.09,79.44,94.11,23.70,50.48,...,75.75,54.44,16.07,84.62,62.25,36.19,21.10,34.28,52.76,10139.71
1,2004-07-30,82.36,50.25,56.88,52.3474,50.75,73.49,95.65,20.92,43.86,...,72.71,52.15,13.01,85.01,62.90,38.54,20.15,31.96,53.01,10173.92
2,2004-08-31,82.36,50.02,59.29,58.8130,52.22,72.70,97.50,18.76,44.71,...,75.31,55.97,13.00,44.98,66.13,39.25,17.89,32.67,52.67,10080.27
3,2004-09-30,79.97,51.46,56.81,79.5303,51.62,80.45,53.64,18.10,40.05,...,78.80,54.12,15.63,43.33,73.74,39.38,18.75,30.60,53.20,10027.47
4,2004-10-29,77.57,53.07,56.80,101.7654,49.90,80.54,53.06,19.21,40.66,...,81.31,51.18,20.32,44.79,72.40,39.10,18.66,28.95,53.92,10428.02


Build diff weight stragy

In [84]:
# HP_DJ_Sto.iloc[0,1:].sum()
# HP_DJ_Sto.iloc[0,0]
# HP_DJ_Sto.iloc[0,1:-1].shape[0]

In [236]:
import pandas as pd
import numpy as np
Portfolio = []
intern = []
number_of_stock = HP_DJ_Sto.iloc[0,1:-1].shape[0]

#value
for i in range(1,HP_DJ_Sto.shape[1]-1):
    intern.append(HP_DJ_Sto.iloc[0,i]/HP_DJ_Sto.iloc[0,1:-1].sum())
Portfolio.append(intern) 

#equal_weight
intern = [1/number_of_stock for _ in range(number_of_stock)]
Portfolio.append(intern) 

#min_var
cov_matrix = HP_DJ_Sto.iloc[:, 1:-1].cov()
one = np.ones(cov_matrix.shape[0])
Inv_cov = np.linalg.inv(cov_matrix)

# min_var_weights = Inv_cov.dot(one) / np.dot(one.T, Inv_cov.dot(one))
min_var_weights = np.dot(Inv_cov, one) / \
                  np.dot(one.T, Inv_cov.dot(one))
min_var_portfolio = np.dot(min_var_weights.T, np.dot(cov_matrix, min_var_weights))
# print(min_var_portfolio)
Portfolio.append(min_var_weights.tolist())

#naive risk parity
vol = HP_DJ_Sto.iloc[:,1:-1].pct_change().std()*np.sqrt(12)
inverse_vol = 1 / vol
normalized_I_vol = [x / sum(inverse_vol) for x in inverse_vol]
Portfolio.append(normalized_I_vol)

#Tangent --- ost Diversified Portfolio
ret_matix = ret_matix = HP_DJ_Sto.iloc[:,1:-1].pct_change().mean()
tangent_weight = np.dot(Inv_cov,ret_matix)/ \
                np.dot(one.T,Inv_cov.dot(ret_matix))
Portfolio.append(tangent_weight)

# display(weight)


In [80]:
# HP_DJ_Sto.iloc[:,1:-1].pct_change()
# one = np.ones(cov_matrix.shape[0])
# ret_matix = HP_DJ_Sto.iloc[:,1:-1].pct_change().mean()
# HP_DJ_Sto.iloc[:,1:-1]

In [237]:
#buy largest
HP_DJ_Sto_Ind_ori = helper.load_data('HW_DJIA_Prices.xlsx')
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind_ori.copy()
industries = HP_DJ_Sto_Ind.iloc[0,1:-1]
# Drop the first row to retain only stock prices
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind.iloc[[4]]
largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax()
# largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax(axis = 1)
weight_of_buylarge = 1/largest_stocks.shape[1]
largest_stocks_weights=[]

for stock in HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns:
    if stock in largest_stocks.iloc[0].values:
        largest_stocks_weights.append(weight_of_buylarge)
    else:
            largest_stocks_weights.append(0)
Portfolio.append(largest_stocks_weights)

C:\Users\zhaoj\AppData\Local\Temp\ipykernel_62112\2927928264.py:7: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax()


In [231]:
# for stock in HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns:
#     print(stock)
# print("\n")
# for i in largest_stocks.iloc[0]:
#     print(i)
# largest_stocks.iloc[[0]]

In [287]:
#idustry_equal
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind_ori.copy()
industries = HP_DJ_Sto_Ind.iloc[0,1:-1]
# HP_DJ_Sto_Ind = HP_DJ_Sto_Ind.iloc[1]

number_ind = HP_DJ_Sto_Ind.groupby(industries, axis=1).count().iloc[0]
equal_weights = number_ind.loc[HP_DJ_Sto_Ind_ori.iloc[0,1:-1]].apply(lambda x:1/x*0.2).values()

C:\Users\zhaoj\AppData\Local\Temp\ipykernel_62112\4010919648.py:6: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  number_ind = HP_DJ_Sto_Ind.groupby(industries, axis=1).count().iloc[0]


TypeError: 'numpy.ndarray' object is not callable

In [288]:
number_ind.loc[HP_DJ_Sto_Ind_ori.iloc[0,1:-1]].apply(lambda x:1/x*0.2).value


AttributeError: 'Series' object has no attribute 'value'

In [266]:
# weight_DF = pd.DataFrame(Portfolio,index = ['value','equal_weight','min_var','naive','tangent','large5','idustry_equal'], columns = HP_DJ_Sto.columns[1:])
# weight_DF

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
value,0.058849,0.033593,0.035678,0.032092,0.033403,0.051938,0.061530,0.015495,0.033004,0.016666,...,0.018673,0.049526,0.035593,0.010507,0.055325,0.040699,0.023661,0.013795,0.022412,0.034495
equal_weight,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,...,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333
min_var,-0.054223,-0.019520,0.006407,0.000233,-0.047194,-0.025272,-0.032463,0.204430,0.022964,-0.036486,...,-0.092853,-0.004810,-0.103891,0.005730,-0.004574,-0.008195,0.098050,0.131386,0.513913,0.095976
naive,0.039922,0.024584,0.032634,0.024223,0.024914,0.024308,0.030633,0.030877,0.039531,0.032606,...,0.035466,0.025778,0.052461,0.018902,0.018718,0.029704,0.046836,0.025138,0.038681,0.046845
tangent,-0.071317,0.024424,0.058171,-0.001104,-0.065691,-0.047037,-0.036765,0.433695,-0.049539,-0.080224,...,-0.010301,0.002001,-0.239466,0.033192,0.015856,-0.010190,0.058627,0.220230,0.283659,0.055307
large5,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000


A    3
B    0
C    2
dtype: int64

In [172]:
# len(Portfolio[4])
# weight = pd.DataFrame(data = Portfolio,index = ['value','equal_weight','min_var','naive','tangent','large5','idustry_equal'], columns = HP_DJ_Sto.columns[1:])
# HP_DJ_Sto_Ind
# largest_stocks_weights
# HP_DJ_Sto_Ind.index
# largest_stocks
# print(HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns)
# largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax()
# print(largest_stocks.iloc[0,1])
# for stock in largest_stocks:
# # HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns:
#     print(stock)
# weight

GOLDMAN SACHS GROUP INC
Energy and Transportation
Finance
Manufacturing
Technology
Trade and Services
